<a href="https://colab.research.google.com/github/tatsath/Interpretability/blob/main/Intervention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This cookbook provides some examples of how to use features and steering in Goodfire in sometimes non-traditional ways.

Such as:
- Dynamic prompts
- Removing Knowledge
- Sorting by features
- On-demand RAG


In [ ]:
!pip install goodfire --quiet
!pip install datasets --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 970.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.6 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

# Add your Goodfire API Key to your Colab secrets
GOODFIRE_API_KEY = userdata.get('GOODFIRE_API_KEY')

## Initialize the SDK

In [ ]:
import goodfire

client = goodfire.Client(GOODFIRE_API_KEY)

# Instantiate a model variant
variant = goodfire.Variant("meta-llama/Meta-Llama-3.1-8B-Instruct")

## Removing knowledge

Let's say we want a model to not know anything about famous people so that we don't get in trouble if it says bad things about them.

We'll use feature search to find features that are relevant to famous people and then play with what happens.

In [ ]:
famous_people_features = client.features.search("celebrities", model=variant, top_k=10)
print(famous_people_features)

After some experimentation, we found a set of feature edits that make the model still recognize celebrity names as noteworthy individuals but forgets all personal details about them.

In [ ]:
variant.reset()
variant.set(famous_people_features[1], -0.5)
variant.set(famous_people_features[9], -0.5)

for token in client.chat.completions.create(
    [
        {"role": "user", "content": "Who is Brad Pitt?"}
    ],
    model=variant,
    stream=True,
    max_completion_tokens=150,
):
    print(token.choices[0].delta.content, end="")

## Dynamic Prompts

In this example, we'll create a model variant that responds to the user's prompt with a different response depending on whether the user is asking for code or not.

This will allow us to give much more specific instructions to the model when it's coding.

### Find Programming Features

We'll first find features that are relevant to programming. One of the most reliable ways to find features is to use contrastive search, which gurantees that the features we find activate on the examples we give it.

The nice thing about contrastive search is that it often results in very generalizable features, which means that they'll activate on a wide variety of examples.


In [ ]:
variant.reset()

_, programming_features = client.features.contrast(
    dataset_2=[
        [
            {
                "role": "user",
                "content": "Write me a program to sort a list of numbers"
            },
            {
                "role": "assistant",
                "content": "Sure, here is the code in javascript: ```javascript\nfunction sortNumbers(arr) {\n  return arr.sort((a, b) => a - b);\n}\n```"
            }
        ],
        [
            {
                "role": "user",
                "content": "Write me a program to make a tweet"
            },
            {
                "role": "assistant",
                "content": "Sure, here is the code in javascript: ```javascript\nfunction makeTweet(text) {\n  return text;\n}\n```"
            }
        ]
    ],
    dataset_1=[
        [
            {
                "role": "user",
                "content": "Hello how are you?"
            },
            {
                "role": "assistant",
                "content":
                  "I'm doing well!"
            },
        ], [
            {
                "role": "user",
                "content": "What's your favorite food?"
            },
            {
                "role": "assistant",
                "content":
                  "It's pizza!"
            },
        ]
    ],
    model=variant,
    top_k=30
)

programming_features = client.features.rerank(
    features=programming_features,
    query="programming",
    model=variant,
    top_k=5
)

print(programming_features)

# Feature # 3 is: "The user is requesting code to be written or generated"
request_programming_feature = programming_features[2]

Next we'll use the features.inspect endpoint to check if the model is requesting code. features.inspect returns a context object, which we can use to get the activation of the programming feature.

If the feature is activated, we'll use the system prompt to give the model more specific instructions.

If the feature is not activated, we'll use the default system prompt.

Without the dynamic prompt, llama 8B tends to write less detailed code with more TODOs and fewer useful comments.

In [ ]:


def check_if_requesting_programming(prompt):
    variant.reset()
    context = client.features.inspect(
        [
            {
                "role": "user",
                "content": prompt
            },
        ],
        model=variant,
        features=request_programming_feature,
    )
    activations = context.top(k=1)
    highest_activation = max(activations, key=lambda x: x.activation)
    return highest_activation.activation > 0.5 #this threshold is arbitrary, but it's a good starting point


def generate_response(prompt):

    is_requesting_programming = check_if_requesting_programming(prompt)
    system_prompt = "You are a helpful assistant."
    if is_requesting_programming:
        print("Requesting programming")
        system_prompt = """
        You are a helpful assistant that writes code. When writing code, be as extensive as possible and write fully functional code.
        Always include comments and proper formatting.
        NEVER leave 'todos' or 'placeholders' in your code.
        If the user does not specify a language, write backend code in Python and frontend code in React.
        Do not explain what your code does, unless the user asks. Just write it.
        """

    for token in client.chat.completions.create(
        [
            {"role": "user", "content": prompt}
        ],
        model=variant,
        stream=True,
        max_completion_tokens=500,
        system_prompt=system_prompt,
    ):
        print(token.choices[0].delta.content, end="")

generate_response("Write me a program to sort a list of numbers")


## Sort by features

You can use feature activations as a way to filter and sort data. In this case let's find some of Elon Musk's tweets that are sarcastic.

In [ ]:
from datasets import load_dataset
num_train_samples = 100
elon_tweets = load_dataset("lcama/elon-tweets", split="train[0:100]")
elon_tweets = elon_tweets.select(range(num_train_samples))
elon_tweets


In [ ]:
sarcasm_features = client.features.search("sarcasm in tweets", model=variant, top_k=4)
print(sarcasm_features)


Find all tweets with a sarcasm score > 1

In [ ]:
def score_sarcasm_on_tweet(tweet):
    context = client.features.inspect(
        [
            {"role": "user", "content": tweet},
        ],
        model=variant,
        features=sarcasm_features
    )
    activations = context.top(k=len(sarcasm_features))
    total_activation = sum(activation.activation for activation in activations)
    return total_activation


tweets_list = list(elon_tweets)
# get any tweets with sarcasm > 1
sarcastic_tweets = [tweet for tweet in tweets_list if score_sarcasm_on_tweet(tweet["text"]) > 1]
sarcastic_tweets


## On-Demand RAG

If we see the user is asking about something that might need more data, e.g. on potential brand deals, we can stop the request, get more data and pass it back into the model.

For example, if the user asks about drinks, and we sponsor Coca Cola, we can stop the request, get RAG data on brand deals and pass it back into the model.

In [ ]:
consumerism_features = client.features.search("coca cola", model=variant, top_k=10)
print(consumerism_features)


In [ ]:
def get_rag_data(prompt):
    #this is where you would perform actual RAG search
    return "We have a brand deal with Coca Cola. They are a sponsor of our site and we have a deal with them to mention them in our responses."

def generate_response(prompt):

    variant.reset()
    variant.abort_when(consumerism_features[0] > 0.25)


    generated_tokens = ""
    try:
        for token in client.chat.completions.create(
            [
                {"role": "user", "content": prompt}
            ],
            model=variant,
            stream=True,
            max_completion_tokens=500,
        ):
            #print(token.choices[0].delta.content, end="")
            generated_tokens += token.choices[0].delta.content

        # If we never get to the brand deal, we'll just return the generated tokens
        print(generated_tokens)

    except Exception as e:
        print(e)
        rag_data = get_rag_data(prompt)
        print(generated_tokens)
        variant.reset()
        print("NEW TOKENS")
        for token in client.chat.completions.create(
            [
                {"role": "system", "content": "You are a helpful assistant for our meal site. You have access to the following information on brand deals:" + rag_data},
                {"role": "user", "content": prompt},
                {"role": "assistant", "content": generated_tokens},
            ],
            model=variant,
            stream=True,
            max_completion_tokens=500,
        ):

            print(token.choices[0].delta.content, end="")

    return None

generate_response("What's are some good drinks to pair with pizza?")